<a href="https://colab.research.google.com/github/gomlfx/apiaryfund/blob/master/RandomForestRegressor_risk_reward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install autosklearn
#!pip install pandas_ta

In [2]:
#import autosklearn
import matplotlib.pyplot as plt
import pandas as pd
import lightgbm as lgb
import numpy as np
from lightgbm import LGBMRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("EURUSD_Daily_197101040000_202009110000.csv", sep='\t')
print(df.head(1))
print(df.tail(1))

       <DATE>  <OPEN>  <HIGH>   <LOW>  <CLOSE>  <TICKVOL>  <VOL>  <SPREAD>
0  1971.01.04  0.5369  0.5369  0.5369   0.5369          1      0        50
           <DATE>   <OPEN>   <HIGH>  ...  <TICKVOL>  <VOL>  <SPREAD>
12801  2020.09.11  1.18145  1.18739  ...      46151      0         1

[1 rows x 8 columns]


In [4]:
#from pandas import to_datetime
#df['<DATE>'] = to_datetime(df['<DATE>'])
#df.tail()
df['<DATE>'] = df['<DATE>'].str.replace('.', '')

In [5]:
df['shifted_high'] = df['<HIGH>'].shift(periods=-1)
df['shifted_close'] = df['<CLOSE>'].shift(periods=-1)
df['shifted_low'] = df['<LOW>'].shift(periods=-1)

df = df.rename(columns={"<DATE>":"date",'<OPEN>':'open','<HIGH>':'high','<LOW>':'low','<CLOSE>':'close','<TICKVOL>':'tickvol','<VOL>':'volume'})
df.tail()

,date,open,high,low,close,tickvol,volume,<SPREAD>,shifted_high,shifted_close,shifted_low
12797,20200907,1.18458,1.18485,1.18111,1.18136,31385,0,1,1.18273,1.17739,1.17654
12798,20200908,1.18136,1.18273,1.17654,1.17739,56953,0,1,1.18337,1.18027,1.17528
12799,20200909,1.17747,1.18337,1.17528,1.18027,50318,0,1,1.19174,1.18146,1.17994
12800,20200910,1.18026,1.19174,1.17994,1.18146,60593,0,1,1.18739,1.18459,1.18086
12801,20200911,1.18145,1.18739,1.18086,1.18459,46151,0,1,NaN,NaN,NaN


In [6]:
#standardize: o-c, o-h, o-l
df['st_high'] = df['high'] - df['open']
df['st_close'] = df['close'] - df['open']
df['st_low'] = df['low'] - df['open']

#to try to standardize
df['st_open'] = 0 

df['sh_st_high'] = df['st_high'].shift(periods=-1)
df['sh_st_close'] = df['st_close'].shift(periods=-1)
df['sh_st_low'] = df['st_low'].shift(periods=-1)




In [7]:
# from sklearn.preprocessing import MinMaxScaler
# #scale the training df
# scaler = MinMaxScaler()
# scaled = scaler.fit_transform(df)
# scaled_train = pd.DataFrame(scaled)
# scaled_train

In [8]:
#current biggest extrema is high(0) or low(0)
conditions = [
              (df['st_high'] > df['st_low'].multiply(-1)),
              (df['st_high'] < df['st_low'].multiply(-1)),
             ]
values = [1,0]
df['current_ext'] = np.select(conditions,values)

#predicted biggest extrema is high(1) or low(0)
conditions = [
              (df['sh_st_high'] > df['sh_st_low'].multiply(-1)),
              (df['sh_st_high'] < df['sh_st_low'].multiply(-1)),
             ]
values = [1,0]
df['prob_sh_ext'] = np.select(conditions,values)

In [9]:
df = df.head(-9)
df.tail()

,date,open,high,low,close,tickvol,volume,<SPREAD>,shifted_high,shifted_close,shifted_low,st_high,st_close,st_low,st_open,sh_st_high,sh_st_close,sh_st_low,current_ext,prob_sh_ext
12788,20200825,1.17871,1.18436,1.17840,1.18343,47918,0,1,1.18391,1.18298,1.17721,0.00565,0.00472,-0.00031,0,0.00051,-0.00042,-0.00619,1,0
12789,20200826,1.18340,1.18391,1.17721,1.18298,44133,0,1,1.19013,1.18217,1.17621,0.00051,-0.00042,-0.00619,0,0.00716,-0.00080,-0.00676,0,1
12790,20200827,1.18297,1.19013,1.17621,1.18217,66444,0,1,1.19196,1.19055,1.18110,0.00716,-0.00080,-0.00676,0,0.00981,0.00840,-0.00105,1,1
12791,20200828,1.18215,1.19196,1.18110,1.19055,62437,0,1,1.19660,1.19361,1.18839,0.00981,0.00840,-0.00105,0,0.00624,0.00325,-0.00197,1,1
12792,20200831,1.19036,1.19660,1.18839,1.19361,50159,0,1,1.20108,1.19113,1.19011,0.00624,0.00325,-0.00197,0,0.00750,-0.00245,-0.00347,1,1


In [10]:
#find which extrema is predicted to hit for the day
clf = RandomForestClassifier(n_estimators=1000)

features = ['open','high','low','close','current_ext']
target = 'prob_sh_ext'
train = df

clf.fit(train[features],train[target])
print('For risk:reward predicted extrema:')
predicted = clf.predict([[1.19358,1.20108,1.19011,1.19113,1]]) 
print(predicted)

For risk:reward predicted extrema:
[0]


In [12]:
#fit df to model, find exact value for classifier prediction
model = RandomForestRegressor(n_estimators=1000)
features = ['open','high','low','close','current_ext']
target1 = 'shifted_high'
target2 = 'shifted_close'
target3 = 'shifted_low'
train = df

# model.fit(train[features],train[target1])
# print('For risk:reward predicted high:')
# predicted = model.predict([[1.19358,1.20108,1.19011,1.19113,1]]) 
# print(predicted)

model.fit(train[features],train[target3])
print('For risk:reward predicted low:')
predicted = model.predict([[1.19358,1.20108,1.19011,1.19113,1]]) 
print(predicted)

For risk:reward predicted low:
[1.18656639]
